In [8]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import telegram
import pandahouse as ph
from datetime import date, timedelta, datetime
import io
from read_db.CH import Getch
import sys
import os


bot = telegram.Bot(token='2096540297:AAGK5o-a19G558tYZ5ck6XgVSMAHd7z6MA4')
chat_id = 478192804

two_days_ago = date.today() - timedelta(days=2)
two_days_ago = datetime.strftime(two_days_ago, '%Y-%m-%d')

metrics = ['sum_views', 'users_lenta', 'sum_likes', 'users_messages', 'sum_mesag', 'CTR' ]

In [11]:
def get_data():
    
    connection = {
    'host': 'https://clickhouse.lab.karpov.courses',
    'password': 'dpo_python_2020',
    'user': 'student',
    'database': 'simulator'}
    
    sql_query = ''' SELECT 
  ts, date, hm, sum_views, users_lenta, sum_likes, users_messages, sum_mesag, CTR
FROM
(SELECT
toStartOfFifteenMinutes(time) as ts,
toDate(ts) as date,
formatDateTime(ts, '%R') as hm,
uniqExact(user_id) as users_lenta,
countIf(action, action = 'view') as sum_views,
countIf(action, action = 'like') as sum_likes,
countIf(action, action = 'like') / countIf(action, action = 'view') *100 as CTR
FROM simulator_20220120.feed_actions 
WHERE ts >=  today() - 7 and ts < toStartOfFifteenMinutes(now())
GROUP BY ts, date, hm
) as l
JOIN
  (SELECT
  toStartOfFifteenMinutes(time) as ts,
  toDate(ts) as date,
  formatDateTime(ts, '%R') as hm,
  uniqExact(user_id) as users_messages,
  count(user_id) as sum_mesag 
  FROM simulator_20220120.message_actions 
  WHERE ts >=  today() - 7 and ts < toStartOfFifteenMinutes(now())
  GROUP BY ts, date, hm
  ) as r
ON r.ts = l.ts 

ORDER by ts '''
    
    data = ph.read_clickhouse(sql_query, connection=connection) 



    return data

In [3]:
def check_anomaly(data, metric, threshold=5):

    data['rolling_mean'] = data[metric].rolling(5).mean().shift()
    data['std'] = data[metric].rolling(5).std().shift()
    data['up_bound'] = data['rolling_mean'] + threshold * data['std']
    data['low_bound'] = data['rolling_mean'] - threshold *  data['std']



    current_ts = data['ts'].max()
    current_value = data[data['ts'] == current_ts][metric].iloc[0]
    current_hm = data[data['ts'] == current_ts].hm.iloc[0]
    current_date = data[data['ts'] == current_ts].date.iloc[0]
    std =  data[data['ts'] == current_ts]['std'].iloc[0]


    mean = data[data['ts'] == current_ts].rolling_mean.iloc[0]
    up_bound = data[data['ts'] == current_ts].up_bound.iloc[0]
    low_bound = data[data['ts'] == current_ts].low_bound.iloc[0]

    anomalies = (current_value > up_bound) or (current_value < low_bound)
    change = (current_value - mean) / std

    return anomalies, change

In [4]:
def check_all(metrics, threshold=5):
    for i in metrics:
    
        if check_anomaly(data, i)[0]:
            current = data[i].iloc[-1]
            
            change = check_anomaly(data, i)[1]
            msg = f"Метрика {i} \n Текущее значение {current}. Отклонение {change:.5}std."

            
            
            data['rolling_mean'] = data[i].rolling(5).mean().shift()
            data['std'] = data[i].rolling(5).std().shift()
            data['up_bound'] = data['rolling_mean'] + threshold * data['std']
            data['low_bound'] = data['rolling_mean'] - threshold *  data['std']

            
    
            
            plt.title(f"динамика метрики {i} за последние 2 дня")
            plt.plot(data['rolling_mean'][data.date > two_days_ago], label='Rolling Mean')
            plt.plot(data[i][data.date > two_days_ago], label='Raw Data')
            plt.plot(data['up_bound'][data.date > two_days_ago], label='up bound')
            plt.plot(data['low_bound'][data.date > two_days_ago], label='low bound')


            locs, labels = plt.xticks()
            plt.xticks(locs, data.hm[data.date > two_days_ago][::19].values)

            plt.gcf().autofmt_xdate()
            
            plt.legend()
            plt.ylabel(i)
            plt.xlabel('time')
            plot_object = io.BytesIO()
            plt.savefig(plot_object)
            plot_object.seek(0)
            plot_object.name = 'dynamics.png'





            plt.close()
            bot.sendPhoto(chat_id=chat_id, photo=plot_object)
            bot.sendMessage(chat_id=chat_id, text=msg)
    return
            

In [ ]:
data = get_data()

check_all(metrics)